# Glosario Total Renting

In [1]:
# Cargamos librerias

import pandas as pd
import people_also_ask
from youtubesearchpython import VideosSearch
import re

In [6]:
# Improtamos en un DF los coches para la búsqueda

coches = pd.read_csv(r'C:\Users\mario\OneDrive\Escritorio\Trabajo\Glosario_FQ\Data\coches.csv')
  
coches = list(coches.Keyword.apply(lambda x: x.strip())) # Lo pasamos a una lista y eliminamos espacios

coches[:10]

['Abarth 500',
 'Alfa Romeo Giulia',
 'Alfa Romeo Stelvio',
 'Audi A1',
 'Audi A3',
 'Audi A4',
 'Audi A5',
 'Audi A6',
 'Audi A7',
 'Audi A8']

In [9]:
# Función que devuelve pregunta-respuesta en formato HTML

def obtener_preguntas_frecuentes_html(key_word, n_preguntas=5, search='Buscar'):
    
    '''
    Esta función sirve para sacar las preguntas frecuentes y su respectiva respuesta
    para cualquier key word
    
    Parameters
    ----------
    key_word: string
        key word de la que queremos encontrar las preguntas y respuestas frecuentes 
    
    n_preguntas: int
        número de preguntas máximas que queremos sacar de la key word, por defecto 5
    
    search: string
        palabra 'Buscar' en el idioma que vamos a buscar las palabras frecuentes, por defecto en español
    
    Returns
    -------
    string
        devuelve un string con las preguntas y las respuestas en formato HTML
    '''
    try:
        questions = people_also_ask.get_related_questions(key_word, n_preguntas)
        questions = [question[0:question.find(search)] for question in questions] # Sacamos las preguntas frecuentes limpias

        answers = []
        contenido = []
        for question in questions:
            try:
                if len(people_also_ask.get_answer(question)['response']) > 3 and people_also_ask.get_answer(question)['snippet_type'] == 'Definition Featured Snippet':    # Comprobamos si el tamaño de la respuesta es mayor a 3 y si la respuesta es una definición
                    answers.append(re.sub(r'[0-9]{1,2} [a-z]{3,4} [0-9]{4}', '', people_also_ask.get_answer(question)['response']))                                        # Eliminamos la fecha final de la respuesta                                                                                      # Cogemos la respuesta tal cual sale
                    contenido.append('<h3>' + question + '</h3>' + '<p>' + re.sub(r'[0-9]{1,2} [a-z]{3,4} [0-9]{4}', '', people_also_ask.get_answer(question)['response']) + '</p>')

                # Esta parte es por si la respuesta a la pregunta es una tabla
                elif len(people_also_ask.get_answer(question)['response']) > 3 and people_also_ask.get_answer(question)['snippet_type'] == 'Table Featured Snippet':       # Comprobamos si el tamaño de la respuesta es mayor a 3 y si la respuesta es una tabla
                    columns = ' '.join(people_also_ask.get_answer(question)['snippet_data']['columns'])                                                                    # Cogemos la respuesta de la columna
                    values = ' '.join([' '.join([str(item) for item in sublist]) + '<br>' for sublist in people_also_ask.get_answer(question)['snippet_data']['values']])  # Sacamos los values en una única lista
                    result = columns + '<br>' + values
                    answers.append(result)  
                    contenido.append('<h3>' + question + '</h3>' + '<p>' + result + '</p>')

                # Esta parte es por si la respuesta a la pregunta no es ni una tabla ni definition
                else:
                    answers.append('')
            except:
                answers.append('') # Si no encontramos respuesta añadimos un elemento vacío

        contenido = [''.join(contenido)][0]

        return contenido
    
    except:
        return print('There are no frequently asked questions')

In [19]:
# Función que devuelve pregunta-respuesta en formato DataFrame

def obtener_preguntas_frecuentes(key_word, n_preguntas=5, search='Buscar'):
    
    '''
    Esta función sirve para sacar las preguntas frecuentes y su respectiva respuesta
    para cualquier key word
    
    Parameters
    ----------
    key_word: string
        key word de la que queremos encontrar las preguntas y respuestas frecuentes 
    
    n_preguntas: int
        número de preguntas máximas que queremos sacar de la key word, por defecto 5
    
    search: string
        palabra 'Buscar' en el idioma que vamos a buscar las palabras frecuentes, por defecto en español
    
    Returns
    -------
    df
        devuelve un df de preguntas y respuestas
    '''
    try:
        questions = people_also_ask.get_related_questions(key_word, n_preguntas)
        questions = [question[0:question.find(search)] for question in questions] # Sacamos las preguntas frecuentes limpias

        answers = []
        for question in questions:
            try:
                if len(people_also_ask.get_answer(question)['response']) > 3 and people_also_ask.get_answer(question)['snippet_type'] == 'Definition Featured Snippet':    # Comprobamos si el tamaño de la respuesta es mayor a 3 y si la respuesta es una definición
                    answers.append(re.sub(r'[0-9]{1,2} [a-z]{3,4} [0-9]{4}', '', people_also_ask.get_answer(question)['response']))                                        # Eliminamos la fecha final de la respuesta                                                                                      # Cogemos la respuesta tal cual sale

                # Esta parte es por si la respuesta a la pregunta es una tabla
                elif len(people_also_ask.get_answer(question)['response']) > 3 and people_also_ask.get_answer(question)['snippet_type'] == 'Table Featured Snippet':       # Comprobamos si el tamaño de la respuesta es mayor a 3 y si la respuesta es una tabla
                    columns = ' '.join(people_also_ask.get_answer(question)['snippet_data']['columns'])                                                                    # Cogemos la respuesta de la columna
                    values = ' '.join([' '.join([str(item) for item in sublist]) for sublist in people_also_ask.get_answer(question)['snippet_data']['values']])  # Sacamos los values en una única lista
                    result = columns + values
                    answers.append(result)

                # Esta parte es por si la respuesta a la pregunta no es ni una tabla ni definition
                else:
                    answers.append('')
            except:
                answers.append('') # Si no encontramos respuesta añadimos un elemento vacío

            # Creamos una lista sólo de las preguntas que tienen respuesta de cada KeyWord
            question_answer = []
            for question, answer in zip(questions, answers):
                if answer != '':
                    question_answer.append([question, answer])
                else:
                    pass
            
            # Desglosamos la lista
            flat_list = []
            for q in question_answer:
                for item in q:
                    flat_list.append(item)
            
            flat_list.insert(len(flat_list), key_word)
        
        return flat_list
    
    except:
        return print('There are no frequently asked questions')

In [22]:
obtener_preguntas_frecuentes('Alfa Romeo Stelvio', 3, 'Buscar')

['¿Cuánto corre el Alfa Romeo?',
 'Modelo Años VelocidadRM 1923-1925 95 km/h 6C-1500 1927-1929 110 km/h 6C-1750 1929-1933 110-170 km/h 8C-2300 1931-1934 165-225 km/h',
 '¿Qué motor monta el Alfa Romeo Stelvio?',
 'La gama de motores de gasolina del Alfa Romeo Stelvio también dispone de dos motores de cuatro cilindros y turbo, ambos con tracción total, el 2.0 Turbo con 200 CV y el 2.0 Turbo con 280 CV de potencia.',
 '¿Cuál es el Alfa Romeo más rápido?',
 'Con un tiempo de 7 minutos, 51 segundos y 7 décimas, el Alfa Romeo Stelvio Quadrifoglio Verde no es solo el SUV más rápido del mundo, sino que ha conseguido batir a berlinas y deportivos de renombre. Sin duda, se ha convertido en el nuevo referente de los SUV compactos premium.',
 '¿Cuánto vale una camioneta Alfa Romeo?',
 'Versión CV PrecioALFA ROMEO STELVIO 2.1 TD 140KW SPRINT AT8 Q4 190 58.800€ ALFA ROMEO STELVIO 2.0 T 148KW SPRINT AT8 Q4 200 59.300€ ALFA ROMEO STELVIO 2.2 D TURBO 154KW TI AUTO 4WD 210 62.800€ ALFA ROMEO STELVIO 2.

In [72]:
obtener_preguntas_frecuentes_html('Alfa Romeo Stelvio', 5, 'Buscar')

'<h3>¿Cuántos caballos tiene el Stelvio?</h3><p>La gama diésel del Alfa Stelvio se compone de tres motores de cuatro cilindros, con 160 CV, 190 CV y 210 CV de potencia. El de 160 CV de potencia únicamente está disponible con tracción trasera, siendo el más asequible de la gama.</p><h3>¿Qué motor lleva el Stelvio diesel?</h3><p>Bajo el capó delantero de este Alfa Romeo Stelvio tenemos un motor de casi 2,2 litros diésel sobrealimentado que rinde 210 CV y un par máximo de 470 Nm de par que proporciona un buen empuje y cuyo principal punto débil es que tiene un sonido indigno de un coche italiano con el biscione en la coraza delantera.</p><h3>¿Qué motor monta el Alfa Romeo Stelvio?</h3><p>La gama de motores de gasolina del Alfa Romeo Stelvio también dispone de dos motores de cuatro cilindros y turbo, ambos con tracción total, el 2.0 Turbo con 200 CV y el 2.0 Turbo con 280 CV de potencia.</p><h3>¿Quién fabrica los motores del Stelvio?</h3><p>Ferrari</p>'

In [ ]:
pr = list(map(obtener_preguntas_frecuentes, coches[:2])) # Saca una lista de las listas de cada Key word

In [98]:
pr[0]

,question,answer
0,¿Cuál es el Abarth con más caballos?,Alpha-N acaba de presentar el Abarth 500 EVOX ...
1,¿Cuántos caballos tiene el Abarth 500?,404 CV
2,¿Cuánto cuesta un Abarth 124 Spider?,Precios del Abarth 124 Spider en febrero 2022 ...
3,¿Cuántos caballos tiene el Abarth 595?,180 CV
4,¿Cuál es el Abarth más barato?,El Abarth más barato tiene un precio de 16.100...
5,¿Cuántos CV tiene un Abarth?,Acabados Potencia ConsumoAbarth 595 Cabrio 201...
